In [21]:
from yaml import load, dump
import json

JOB_NAME = "cargo_doc"
INPUT_FILE = "github/" + JOB_NAME + ".yml"
OUTPUT_FILE = "Outputs/github/" + JOB_NAME + ".json"
OUTPUT_JOB = "Outputs/gitlab/" + JOB_NAME + ".yml"

try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
    

In [22]:
yaml_file = open(INPUT_FILE, 'r')
yaml_content = load(yaml_file, Loader=Loader)
# with open(OUTPUT_FILE, "w") as outfile:
#     json.dump(yaml_content, outfile)
# print("Key: Value")
# for key, value in yaml_content.items():
#     print(f"{key}: {value}")

In [23]:
def browseDict(keyElt, dictionary):
    NOT_FOUND = ""
    for key, value in dictionary.items():        
        if (key == keyElt):
            return (keyElt, value)
        if (isinstance(value,dict)):
            result = browseDict(keyElt, value)
            if(result != NOT_FOUND):
                return result
    return NOT_FOUND           

In [24]:
def get_variables(dictionary):
    variables = {}
    _, value = browseDict("workflow_call", dictionary)
    inputs = value.get("inputs", "")
    secrets = value.get("secrets", "")
    if(inputs):
        for key, value in inputs.items():
            variables[key] = value.get("default", "")
    if(secrets):
        for key, value in secrets.items():
            variables[key] = value.get("default", "")
    return variables

#get_variables(yaml_content)

In [12]:
def get_docker_image(job):
    container = job.get("container", "")
    if(container):
        if(isinstance(container, str)):
            return container
        elif(isinstance(container, dict)):
            return container.get("image", "")
    return ""
    
def get_runs_on(job):
    return job.get("runs-on", "")

def get_steps(job):
    return job.get("steps", [])

def get_artifacts(steps):
    artifacts = {}
    for elt in steps:
        if "actions/upload-artifact" in elt.get("uses", ""):
            artifacts["name"] = elt["with"].get("name", "")
            artifacts["paths"] = elt["with"].get("path", "").splitlines()
            artifacts["expire_in"] = elt["with"].get("retention-days", "")
            when = elt.get("if", "")
            if when == "failure()":
                artifacts["when"] = "on_failure"
            elif when == "always()":
                artifacts["when"] = "always"
            else:
                artifacts["when"] = "on_success"
                
    return artifacts
    
def get_cache(steps):
    cache = {}
    for elt in steps:
        if "actions/cache" in elt.get("uses", ""):
            cache["key"] = elt["with"].get("key", "")
            cache["paths"] = elt["with"].get("path", "").splitlines()
    return cache

def get_script(steps):
    script = []
    for step in steps:
        if "run" in step:
            lines = step.get("run").splitlines()
            for line in lines:
                script.append(line)
    return script
     

In [26]:
# from os.path import basename, splitext
# from collections import OrderedDict
_,jobs = browseDict("jobs", yaml_content)
gitlab_cicd_dict = {}
variables = get_variables(yaml_content)
for job_name, job in jobs.items():
    gitlab_cicd_dict[job_name] = {}
    image = get_docker_image(job)
    if image:
        gitlab_cicd_dict[job_name]["image"] = image
    gitlab_cicd_dict[job_name]["variables"] = variables
    steps = get_steps(job)
    cache = get_cache(steps)
    if cache:
        gitlab_cicd_dict[job_name]["cache"] = cache
    script = get_script(steps)
    gitlab_cicd_dict[job_name]["script"] = script
    artifacts = get_artifacts(steps)
    if artifacts:
        gitlab_cicd_dict[job_name]["artifacts"] = artifacts

with open(OUTPUT_JOB, 'w') as job_file:
    dump(gitlab_cicd_dict, job_file)
yml_result = dump(gitlab_cicd_dict, Dumper=Dumper)
print(yml_result)
            


cargo_doc:
  artifacts:
    expire_in: 15
    name: Documentation
    paths:
    - ${{ env.OUTPUT_FOLDER }}
    when: on_success
  image: rust:1.57-buster
  script:
  - echo "CIWORKSPACE_PATH=$GITHUB_WORKSPACE" >> $GITHUB_ENV
  - cd ${{ inputs.PROJECT_ROOT }}
  - echo "PROJECT_ROOT_RELATIVE_PATH=$(pwd)" >> $GITHUB_ENV
  - 'echo "GITHUB_WORKSPACE : $GITHUB_WORKSPACE" '
  - cd ${{ inputs.PROJECT_ROOT }}
  - if [ ! -f "Cargo.toml" ]; then
  - '  echo "ERROR --> Any Cargo.toml file isn''t present in the given root project
    folder ${{ inputs.PROJECT_ROOT }}"'
  - '  echo "INSIDE_IF_PATH=$(pwd)" >> $GITHUB_ENV'
  - fi
  - echo "ADDITIONAL_OPTIONS=${{inputs.ADDITIONAL_OPTIONS}}" >> $GITHUB_ENV
  - echo "OUTPUT_FOLDER=${{inputs.OUTPUT_FOLDER}}" >> $GITHUB_ENV
  - if [ ${{ inputs.ONLY_LIB }} ]; then
  - '  echo "ADDITIONAL_OPTIONS=${{env.ADDITIONAL_OPTIONS}} --lib" >> $GITHUB_ENV'
  - fi
  - if [ ${{ inputs.RELEASE_MODE }} ]; then
  - '  echo "ADDITIONAL_OPTIONS=${{env.ADDITIONAL_OPTIONS}} -

In [13]:
jobs_list = []
_,jobs = browseDict("jobs", yaml_content)
for key, value in jobs.items():
    jobs_list.append((key, value))
_, job = jobs_list[0]
jobs_list[0]

('reusable-cypress_run',
 {'runs-on': 'ubuntu-latest',
  'steps': [{'uses': 'actions/checkout@v2'},
   {'name': 'Cache node modules',
    'uses': 'actions/cache@v2',
    'env': {'cache-name': 'cache-node-modules'},
    'with': {'path': '~/.npm\n~/.cache/Cypress\n',
     'key': "${{runner.os}}-build-${{env.cache-name}}-${{hashFiles('**/package-lock.json')}}",
     'restore-keys': '${{runner.os}}-build-${{env.cache-name}}-\n'}},
   {'name': 'Install dependencies', 'run': 'npm ci'},
   {'name': 'Install wait-on package', 'run': 'npm i -g wait-on'},
   {'name': 'Make Cypress wait until the server is up and available',
    'run': 'if [ ! -z ${{inputs.CYPRESS_BASE_URL}} ]; then\n    npm start --no-clipboard & wait-on $CYPRESS_BASE_URL\nelse \n    echo "Variable CYPRESS_BASE_URL is MANDATORY. You must fill it !"\n    exit 1\nfi\n'},
   {'name': 'Run Cypress tests',
    'run': 'if [ ! -z ${{secrets.CYPRESS_RECORD_KEY}} ]; then\n    npx cypress run -P ${{inputs.CYPRESS_PROJECT_PATH}} -C ${{inpu

In [11]:
steps = get_steps(job)
get_script(steps)

['npm ci',
 'npm i -g wait-on',
 'if [ ! -z ${{inputs.CYPRESS_BASE_URL}} ]; then',
 '    npm start --no-clipboard & wait-on $CYPRESS_BASE_URL',
 'else ',
 '    echo "Variable CYPRESS_BASE_URL is MANDATORY. You must fill it !"',
 '    exit 1',
 'fi',
 'if [ ! -z ${{secrets.CYPRESS_RECORD_KEY}} ]; then',
 '    npx cypress run -P ${{inputs.CYPRESS_PROJECT_PATH}} -C ${{inputs.CYPRESS_CONFIG_FILE}} -r ${{inputs.CYPRESS_REPORTER}} ${{inputs.ADDITIONAL_OPTIONS}} --record',
 'else',
 '    npx cypress run -P ${{inputs.CYPRESS_PROJECT_PATH}} -C ${{inputs.CYPRESS_CONFIG_FILE}} -r ${{inputs.CYPRESS_REPORTER}} ${{env.ADDITIONAL_OPTIONS}}',
 'fi']

In [81]:
lines = "~/.npm\n~/.cache/Cypress\n".splitlines()
for line in lines:
    print(line)
print("Lines", lines)

Lines []
